In [111]:
import pandas as pd

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Show the full content of each cell
pd.set_option('display.width', None)  # Auto-detect the display width



In [112]:
# reading hospital admission data
df = pd.read_csv('./dataset/mimic-iii-demo-1.4/ADMISSIONS.csv')
df.drop(['row_id', 'deathtime', 'discharge_location', 'insurance', 'language', 'religion', 'marital_status', 'ethnicity', 'hospital_expire_flag', 'has_chartevents_data'], axis=1, inplace=True)

# convert string to datetime
df['admittime'] = pd.to_datetime(df['admittime'])
df['dischtime'] = pd.to_datetime(df['dischtime'])
df['edregtime'] = pd.to_datetime(df['edregtime'])
df['edouttime'] = pd.to_datetime(df['edouttime'])

# sort by subject_id, hadm_id and  admittime
df = df.sort_values(by=['subject_id', 'hadm_id', 'admittime'])

print(df.dtypes)
print(df.shape)
display(df.head(5))

subject_id                     int64
hadm_id                        int64
admittime             datetime64[ns]
dischtime             datetime64[ns]
admission_type                object
admission_location            object
edregtime             datetime64[ns]
edouttime             datetime64[ns]
diagnosis                     object
dtype: object
(129, 9)


,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,edregtime,edouttime,diagnosis
0,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS
1,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,NaT,NaT,HEPATITIS B
2,10013,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,NaT,NaT,SEPSIS
3,10017,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,EMERGENCY,EMERGENCY ROOM ADMIT,2149-05-26 12:08:00,2149-05-26 19:45:00,HUMERAL FRACTURE
4,10019,177759,2163-05-14 20:43:00,2163-05-15 12:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,NaT,NaT,ALCOHOLIC HEPATITIS


In [113]:
# reading admitted patient transfers between care units data
df2 = pd.read_csv('./dataset/mimic-iii-demo-1.4/TRANSFERS.csv')
df2.drop(['row_id', 'dbsource', 'prev_careunit', 'prev_wardid'], axis=1, inplace=True)

# convert string to datetime
df2['intime'] = pd.to_datetime(df2['intime'])
df2['outtime'] = pd.to_datetime(df2['outtime'])

# convert float to int
df2['icustay_id'] = df2['icustay_id'].fillna(0).astype('int64')
df2['los'] = df2['los'].fillna(0)
df2['curr_careunit'] = df2['curr_careunit'].fillna('NA')
df2['curr_wardid'] = df2['curr_wardid'].fillna(0)

# sort by subject_id, hadm_id and  intime
df2 = df2.sort_values(by=['subject_id', 'hadm_id', 'intime'])

print(df2.dtypes)
print(df2.shape)
display(df2.head(3))

subject_id                int64
hadm_id                   int64
icustay_id                int64
eventtype                object
curr_careunit            object
curr_wardid             float64
intime           datetime64[ns]
outtime          datetime64[ns]
los                     float64
dtype: object
(524, 9)


,subject_id,hadm_id,icustay_id,eventtype,curr_careunit,curr_wardid,intime,outtime,los
0,10006,142345,206504,admit,MICU,52.0,2164-10-23 21:10:15,2164-10-25 12:21:07,39.18
1,10006,142345,0,transfer,NA,45.0,2164-10-25 12:21:07,2164-11-01 17:14:27,172.89
2,10006,142345,0,discharge,NA,0.0,2164-11-01 17:14:27,NaT,0.00


In [114]:
# merge ADMISSIONS and TRANSFERS
df_merge = pd.merge(df, df2, on=['subject_id', 'hadm_id'])
print(df_merge.dtypes)
print(df_merge.shape)
display(df_merge.head(3))

subject_id                     int64
hadm_id                        int64
admittime             datetime64[ns]
dischtime             datetime64[ns]
admission_type                object
admission_location            object
edregtime             datetime64[ns]
edouttime             datetime64[ns]
diagnosis                     object
icustay_id                     int64
eventtype                     object
curr_careunit                 object
curr_wardid                  float64
intime                datetime64[ns]
outtime               datetime64[ns]
los                          float64
dtype: object
(524, 16)


,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,edregtime,edouttime,diagnosis,icustay_id,eventtype,curr_careunit,curr_wardid,intime,outtime,los
0,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,206504,admit,MICU,52.0,2164-10-23 21:10:15,2164-10-25 12:21:07,39.18
1,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,transfer,NA,45.0,2164-10-25 12:21:07,2164-11-01 17:14:27,172.89
2,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,discharge,NA,0.0,2164-11-01 17:14:27,NaT,0.00


In [115]:
# reading services allocated to admitted patients
df3 = pd.read_csv('./dataset/mimic-iii-demo-1.4/SERVICES.csv')
df3.drop(['row_id'], axis=1, inplace=True)

# convert string to datetime
df3['transfertime'] = pd.to_datetime(df3['transfertime'])

# handle missing values
df3['prev_service'] = df3['prev_service'].fillna('NA')

# sort by subject_id, hadm_id and  transfertime
df3 = df3.sort_values(by=['subject_id', 'hadm_id', 'transfertime'])

print(df3.dtypes)
print(df3.shape)
display(df3.head(3))

subject_id               int64
hadm_id                  int64
transfertime    datetime64[ns]
prev_service            object
curr_service            object
dtype: object
(163, 5)


,subject_id,hadm_id,transfertime,prev_service,curr_service
0,10006,142345,2164-10-23 21:10:15,NA,MED
1,10011,105331,2126-08-14 22:34:00,NA,MED
2,10013,165520,2125-10-04 23:38:00,NA,MED


In [116]:
# merge ADMISSIONS, TRANSFERS and SERVICES
df_merge2 = pd.merge_asof(df_merge.sort_values('intime'), 
                          df3.sort_values('transfertime'), 
                          by=['subject_id', 'hadm_id'], 
                          left_on='intime',
                          right_on='transfertime',
                          direction='nearest',
                          tolerance=pd.Timedelta('10 minutes'))

# tolerance limit problem - 10 minutes seems to have less amount of conflicts. but there are still cases where intime of between two transfers is small and so service is not assigned correctly. there is also the opposite problem that sometimes a service start time does not find any nearset match in 10 minutes range, and ends up getting ignored. this is specially a problem when the transfer has no outtime - that is the discharge case. 

temp = []
flag = True
# fix transfer time wrong alignment
for index in df_merge2.index:
    if len(temp) > 0:
        current = temp.pop(0)
        flag = False
    else:
        current = df_merge2.iloc[index]
    next = df_merge2.iloc[index+1] if index+1 < df_merge2.shape[0] else None
    prev = df_merge2.iloc[index-1] if index-1 >= 0 else None

    if flag:
        if (current['intime'] <= current['transfertime'] <= current['outtime']):
            continue
        elif (current['transfertime'] is pd.NaT) and (current['subject_id'] == prev['subject_id']) and (current['hadm_id'] == prev['hadm_id']):
            df_merge2.at[index, 'transfertime'] = prev['transfertime']
            df_merge2.at[index, 'prev_service'] = prev['prev_service']
            df_merge2.at[index, 'curr_service'] = prev['curr_service']
        elif (current['transfertime'] > next['intime']):
            print(f'index {index} has problem')
            curr_sub_id = current['subject_id'] 
            curr_hadm_id = current['hadm_id'] 
            curr_transfertime = current['transfertime']
            print('current transfer time is greater than next intime') 
            print(f'current: subject_id={curr_sub_id}, hadm_id={curr_hadm_id}, transfertime={curr_transfertime}')
        elif (current['transfertime'] <= next['intime']) and (current['subject_id'] == next['subject_id']) and (current['hadm_id'] == next['hadm_id']) and (current['subject_id'] == prev['subject_id']) and (current['hadm_id'] == prev['hadm_id']):
            df_merge2.at[index, 'transfertime'] = prev['transfertime']
            df_merge2.at[index, 'prev_service'] = prev['prev_service']
            df_merge2.at[index, 'curr_service'] = prev['curr_service']

            if pd.isnull(next['transfertime']):
                df_merge2.at[index+1, 'transfertime'] = current['transfertime']
                df_merge2.at[index+1, 'prev_service'] = current['prev_service']
                df_merge2.at[index+1, 'curr_service'] = current['curr_service']
            else:
                temp.append(next)
                df_merge2.at[index+1, 'transfertime'] = current['transfertime']
                df_merge2.at[index+1, 'prev_service'] = current['prev_service']
                df_merge2.at[index+1, 'curr_service'] = current['curr_service']
    else:
        if (current['transfertime'] <= next['intime']) and (current['subject_id'] == next['subject_id']) and (current['hadm_id'] == next['hadm_id']) and pd.isnull(next['transfertime']):
            df_merge2.at[index+1, 'transfertime'] = current['transfertime']
            df_merge2.at[index+1, 'prev_service'] = current['prev_service']
            df_merge2.at[index+1, 'curr_service'] = current['curr_service']
            flag = True
        else:
            print(f'index {index} has problem')
            curr_sub_id = current['subject_id'] 
            curr_hadm_id = current['hadm_id'] 
            curr_transfertime = current['transfertime']
            print('temp')
            print(f'current: subject_id={curr_sub_id}, hadm_id={curr_hadm_id}, transfertime={curr_transfertime}')


# print(df_merge2.dtypes)
# print(df_merge2.shape)
# display(df_merge2.head(100))
# display(df_merge2.tail(30))
# display(df_merge2[df_merge2['subject_id'] == 10033])
# display(df_merge2[df_merge2['subject_id'] == 41976])
display(df_merge2[df_merge2['subject_id'] == 44083])
# display(df_merge2[df_merge2['subject_id'] == 42231])

index 58 has problem
current transfer time is greater than next intime
current: subject_id=44083, hadm_id=131048, transfertime=2112-05-23 13:18:18


,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,edregtime,edouttime,diagnosis,icustay_id,eventtype,curr_careunit,curr_wardid,intime,outtime,los,transfertime,prev_service,curr_service
51,44083,125157,2112-05-04 08:00:00,2112-05-11 14:15:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,NaT,NaT,ESOPHAGEAL CA/SDA,0,admit,NA,27.0,2112-05-04 01:45:43,2112-05-04 19:03:39,17.30,2112-05-04 01:45:43,NA,TSURG
52,44083,125157,2112-05-04 08:00:00,2112-05-11 14:15:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,NaT,NaT,ESOPHAGEAL CA/SDA,265615,transfer,SICU,33.0,2112-05-04 19:03:39,2112-05-06 17:21:01,46.29,2112-05-04 01:45:43,NA,TSURG
53,44083,125157,2112-05-04 08:00:00,2112-05-11 14:15:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,NaT,NaT,ESOPHAGEAL CA/SDA,0,transfer,NA,36.0,2112-05-06 17:21:01,2112-05-06 17:32:47,0.20,2112-05-04 01:45:43,NA,TSURG
54,44083,125157,2112-05-04 08:00:00,2112-05-11 14:15:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,NaT,NaT,ESOPHAGEAL CA/SDA,0,transfer,NA,36.0,2112-05-06 17:32:47,2112-05-11 14:14:23,116.69,2112-05-04 01:45:43,NA,TSURG
55,44083,125157,2112-05-04 08:00:00,2112-05-11 14:15:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,NaT,NaT,ESOPHAGEAL CA/SDA,0,discharge,NA,0.0,2112-05-11 14:14:23,NaT,0.00,2112-05-04 01:45:43,NA,TSURG
56,44083,131048,2112-05-22 15:37:00,2112-05-25 13:30:00,EMERGENCY,EMERGENCY ROOM ADMIT,2112-05-22 09:25:00,2112-05-22 17:04:00,SHORTNESS OF BREATH,0,admit,NA,36.0,2112-05-22 15:37:59,2112-05-23 12:32:06,20.90,2112-05-22 15:37:59,NA,TSURG
57,44083,131048,2112-05-22 15:37:00,2112-05-25 13:30:00,EMERGENCY,EMERGENCY ROOM ADMIT,2112-05-22 09:25:00,2112-05-22 17:04:00,SHORTNESS OF BREATH,282640,transfer,CCU,7.0,2112-05-23 12:32:06,2112-05-23 13:15:56,0.73,2112-05-22 15:37:59,NA,TSURG
58,44083,131048,2112-05-22 15:37:00,2112-05-25 13:30:00,EMERGENCY,EMERGENCY ROOM ADMIT,2112-05-22 09:25:00,2112-05-22 17:04:00,SHORTNESS OF BREATH,0,transfer,NA,36.0,2112-05-23 13:15:56,2112-05-23 13:17:31,0.03,2112-05-23 13:18:18,TSURG,CMED
59,44083,131048,2112-05-22 15:37:00,2112-05-25 13:30:00,EMERGENCY,EMERGENCY ROOM ADMIT,2112-05-22 09:25:00,2112-05-22 17:04:00,SHORTNESS OF BREATH,282640,transfer,CCU,7.0,2112-05-23 13:17:31,2112-05-25 14:59:50,49.71,2112-05-23 13:18:18,TSURG,CMED
60,44083,131048,2112-05-22 15:37:00,2112-05-25 13:30:00,EMERGENCY,EMERGENCY ROOM ADMIT,2112-05-22 09:25:00,2112-05-22 17:04:00,SHORTNESS OF BREATH,0,discharge,NA,0.0,2112-05-25 14:59:50,NaT,0.00,2112-05-23 13:18:18,TSURG,CMED


In [117]:
# reading lab events data
df4 = pd.read_csv('./dataset/mimic-iii-demo-1.4/LABEVENTS.csv')

# convert string to datetime
df4['charttime_lab'] = pd.to_datetime(df4['charttime'])

# handle missing values
df4['hadm_id'] = df4['hadm_id'].fillna(0).astype('int64')
# df4['value'] = df4['value'].fillna('NA')

df4.drop(['row_id', 'value', 'valuenum', 'valueuom', 'flag', 'charttime'], axis=1, inplace=True)

# sort by subject_id, hadm_id and  charttime_lab
df4 = df4.sort_values(by=['subject_id', 'hadm_id', 'charttime_lab'])

print(df4.dtypes)
print(df4.shape)
display(df4.head(3))

subject_id                int64
hadm_id                   int64
itemid                    int64
charttime_lab    datetime64[ns]
dtype: object
(76074, 4)


,subject_id,hadm_id,itemid,charttime_lab
0,10006,0,50868,2164-09-24 20:21:00
1,10006,0,50882,2164-09-24 20:21:00
2,10006,0,50893,2164-09-24 20:21:00


In [118]:
# merge ADMISSIONS, TRANSFERS, LABEVENTS

df_merge3 = pd.merge(df_merge, df4, on=['subject_id', 'hadm_id'])
print(df_merge3.dtypes)
print(df_merge3.shape)
display(df_merge3.head(3))

subject_id                     int64
hadm_id                        int64
admittime             datetime64[ns]
dischtime             datetime64[ns]
admission_type                object
admission_location            object
edregtime             datetime64[ns]
edouttime             datetime64[ns]
diagnosis                     object
icustay_id                     int64
eventtype                     object
curr_careunit                 object
curr_wardid                  float64
intime                datetime64[ns]
outtime               datetime64[ns]
los                          float64
itemid                         int64
charttime_lab         datetime64[ns]
dtype: object
(273003, 18)


,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,edregtime,edouttime,diagnosis,icustay_id,eventtype,curr_careunit,curr_wardid,intime,outtime,los,itemid,charttime_lab
0,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,206504,admit,MICU,52.0,2164-10-23 21:10:15,2164-10-25 12:21:07,39.18,50813,2164-10-23 17:33:00
1,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,206504,admit,MICU,52.0,2164-10-23 21:10:15,2164-10-25 12:21:07,39.18,50861,2164-10-23 17:38:00
2,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,206504,admit,MICU,52.0,2164-10-23 21:10:15,2164-10-25 12:21:07,39.18,50862,2164-10-23 17:38:00


In [119]:
# reading input events (administered fluid intakes) data
df5 = pd.read_csv('./dataset/mimic-iii-demo-1.4/INPUTEVENTS_MV.csv')

# convert string to datetime
df5['starttime'] = pd.to_datetime(df5['starttime'])
df5['endtime'] = pd.to_datetime(df5['endtime'])
df5['storetime_input'] = pd.to_datetime(df5['storetime'])

# handle missing values
# df5['icustay_id'] = df5['icustay_id'].fillna(0).astype('int64')

df5.drop(['row_id', 'amount', 'amountuom', 'rate', 'rateuom', 'orderid', 'linkorderid', 'ordercategoryname', 'secondaryordercategoryname', 'ordercomponenttypedescription', 'ordercategorydescription', 'patientweight', 'totalamount', 'totalamountuom', 'isopenbag', 'continueinnextdept', 'cancelreason', 'statusdescription', 'comments_editedby', 'comments_canceledby', 'comments_date', 'originalamount', 'originalrate', 'storetime'], axis=1, inplace=True)

# sort by subject_id, hadm_id and  storetime
df5 = df5.sort_values(by=['subject_id', 'hadm_id', 'storetime_input'])

print(df5.dtypes)
print(df5.shape)
display(df5.head(3))

subject_id                  int64
hadm_id                     int64
icustay_id                  int64
starttime          datetime64[ns]
endtime            datetime64[ns]
itemid                      int64
cgid                        int64
storetime_input    datetime64[ns]
dtype: object
(13224, 8)


,subject_id,hadm_id,icustay_id,starttime,endtime,itemid,cgid,storetime_input
5925,40124,126179,279554,2130-02-04 04:35:00,2130-02-05 08:21:00,225152,19085,2130-02-04 04:36:00
5926,40124,126179,279554,2130-02-04 04:35:00,2130-02-05 08:21:00,220949,19085,2130-02-04 04:36:00
5936,40124,126179,279554,2130-02-04 04:36:00,2130-02-04 10:00:00,222168,19085,2130-02-04 04:36:00


In [120]:
# reading output events (excreted fluid) data
df6 = pd.read_csv('./dataset/mimic-iii-demo-1.4/OUTPUTEVENTS.csv')

# # convert string to datetime
df6['storetime_output'] = pd.to_datetime(df6['storetime'])

# # handle missing values
df6['icustay_id'] = df6['icustay_id'].fillna(0).astype('int64')
# df6['value'] = df6['value'].fillna('NA')

df6.drop(['row_id', 'valueuom', 'value', 'stopped', 'newbottle', 'iserror','storetime', 'charttime'], axis=1, inplace=True)

# # sort by subject_id, hadm_id and  storetime_output
df6 = df6.sort_values(by=['subject_id', 'hadm_id', 'storetime_output'])

print(df6.dtypes)
print(df6.shape)
display(df6.head(3))

subject_id                   int64
hadm_id                      int64
icustay_id                   int64
itemid                       int64
cgid                         int64
storetime_output    datetime64[ns]
dtype: object
(11320, 6)


,subject_id,hadm_id,icustay_id,itemid,cgid,storetime_output
1805,10006,142345,206504,40055,14387,2164-10-24 04:58:00
1806,10006,142345,206504,40069,19150,2164-10-25 03:22:00
1807,10006,142345,206504,40054,19150,2164-10-25 03:22:00
